In [8]:
import boto3
from datetime import datetime
import json
import os
import pandas as pd
import sys


base_dir = os.path.abspath('../')   # 메인 디렉토리 설정
sys.path.append(base_dir)
from src import aws_utils as aws
print(base_dir)

c:\Users\deachul.seo\Desktop\repo\yar_airdna


In [9]:
# SETUP

config = aws.load_config()
aws_access_key_id = config['AWS_ACCESS_KEY_ID']
aws_secret_access_key = config['AWS_SECRET_ACCESS_KEY']
bucket_name = config['BUCKET_NAME']
base_prefix = config['BASE_PREFIX']
data_dir = config['DATA_DIRECTORY']
p_ym = config['P_YM']

output_date = aws.convert_date_format(p_ym)
bucket_dir = base_prefix + output_date

masked_id, masked_key = aws.display_config(aws_access_key_id, aws_secret_access_key)   # 민감정보 가리기

print('----------------------------CONFIG----------------------------')
print(f"AWS_ACCESS_KEY_ID: {masked_id}")
print(f"AWS_SECRET_ACCESS_KEY: {masked_key}")
print(f"BUCKET_NAME: {bucket_name}")
print(f"BUCKET_DIR: {bucket_dir}")
print(f"DATA_DIRECTORY: {data_dir}")
print(f"P_YM: {p_ym}")
print('---------------------------------------------------------------')

start_time = datetime.now()
print("START TIME:", start_time.time())


----------------------------CONFIG----------------------------
AWS_ACCESS_KEY_ID: AKI**************NHT
AWS_SECRET_ACCESS_KEY: RBI**********************************r45
BUCKET_NAME: airdna-prod-data-integration
BUCKET_DIR: yanolja/outbound/June 2024
DATA_DIRECTORY: ../data/
P_YM: 2024-06-01
---------------------------------------------------------------
START TIME: 16:24:29.311518


In [10]:
# AWS S3 스토리지 접속
print("Accessing AWS Storage...")
aws_access_key_id = config['AWS_ACCESS_KEY_ID']
aws_secret_access_key = config['AWS_SECRET_ACCESS_KEY']

s3 = aws.init_s3_client(aws_access_key_id, aws_secret_access_key)
print('Accessed S3 storage: ', s3)


Accessing AWS Storage...
Accessed S3 storage:  <botocore.client.S3 object at 0x00000210E14839D0>


In [54]:
print('Start data download...')

aws.airdna_data_download(s3=s3, base_prefix=base_prefix, bucket_name=bucket_name, ml=output_date)

end_time = datetime.now()
print("END TIME:", start_time.time())

Directory '../data/June 2024' already exists.
['yanolja/outbound/June 2024/', 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv', 'yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_property_file_2024_07_07_02_29_48.csv']
DOWNLOADING FILE: yanolja/outbound/June 2024/Yanolja-PPD-South Korea-CSV_monthly_file_2024_07_07_02_29_51.csv...
